In [20]:
import mysql.connector
import pandas as pd

ojk = mysql.connector.connect(
    host = "database-1.cnu82kme6p4d.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "eda",
    password = "ojk0707",
    database = "ojk"
)
cursor = ojk.cursor()
query = "SELECT * FROM enjoy"
enjoy = pd.read_sql(query, ojk)
enjoy.head()

/tmp/ipykernel_24515/1667690205.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,station,category,type,review,address,latitude,longitude
0,(Jokers)조커스,연신내역,보드카페,1,24,서울 은평구 서오릉로 100 지하1층,37.608935,126.921038
1,[푸에르자 부르타] FB씨어터,서울숲역,공연장,1,762,서울 성동구 고산자로 71 FB씨어터,37.544488,127.035898
2,00의 00,성신여대입구역,미술관,1,32,서울 성북구 성북로2길 8 지하1층,37.588997,127.006390
3,1101 어린이라운지,교대역,"키즈카페,실내놀이터",1,3305,서울 서초구 남부순환로 2406 예술의전당 비타민스테이션 1101 어린이라운지,37.480231,127.013467
4,179아트스튜디오,서울역,"셀프,대여스튜디오",1,22,서울 용산구 두텁바위로 179 2층/3층/4층,37.553534,126.978504


In [21]:
import numpy as np
enjoy['indicator']=np.log(enjoy['review']+1)
enjoy.head()

,name,station,category,type,review,address,latitude,longitude,indicator
0,(Jokers)조커스,연신내역,보드카페,1,24,서울 은평구 서오릉로 100 지하1층,37.608935,126.921038,3.218876
1,[푸에르자 부르타] FB씨어터,서울숲역,공연장,1,762,서울 성동구 고산자로 71 FB씨어터,37.544488,127.035898,6.637258
2,00의 00,성신여대입구역,미술관,1,32,서울 성북구 성북로2길 8 지하1층,37.588997,127.006390,3.496508
3,1101 어린이라운지,교대역,"키즈카페,실내놀이터",1,3305,서울 서초구 남부순환로 2406 예술의전당 비타민스테이션 1101 어린이라운지,37.480231,127.013467,8.103494
4,179아트스튜디오,서울역,"셀프,대여스튜디오",1,22,서울 용산구 두텁바위로 179 2층/3층/4층,37.553534,126.978504,3.135494


In [22]:
import plotly.express as px

fig = px.histogram(enjoy,x='indicator',title='지표값별 놀거리 분포',color_discrete_sequence=['green'])
fig.update_layout(
    xaxis_title='지표값 (log(리뷰수 + 1))',
    yaxis_title='놀거리 수'
)
fig.show()

In [23]:
avg_value_value = enjoy['indicator'].mean()
print("전체 맛집의 평균 평가지수:", avg_value_value)

전체 맛집의 평균 평가지수: 4.784839175958799


In [24]:
import plotly.express as px

# 평균 계산
avg_value_value = enjoy['indicator'].mean()
print("전체 놀거리의 평균 지표값:", avg_value_value)

# 히스토그램 생성
fig = px.histogram(
    enjoy,
    x='indicator',
    title='지표값별 놀거리 분포',
    color_discrete_sequence=['green']
)

# y축 최대값 추정 (bin 수에 맞게 설정)
y_max = enjoy['indicator'].value_counts(bins=50).max()

# 평균선 추가
fig.add_shape(
    type='line',
    x0=avg_value_value,
    x1=avg_value_value,
    y0=0,
    y1=y_max,
    line=dict(color='black', dash='dash')
)

# 평균선 라벨 추가
fig.add_annotation(
    x=avg_value_value,
    y=y_max,
    text=f"평균 지표값: {avg_value_value:.2f}",
    showarrow=False,
    yshift=10,
    font=dict(color='black')
)

# 레이아웃 수정
fig.update_layout(
    xaxis_title='지표값 (log(리뷰수 + 1))',
    yaxis_title='놀거리 수'
)

fig.show()


전체 놀거리의 평균 지표값: 4.784839175958799


In [30]:
# 역별 평균 value 계산
station_indicator_avg = enjoy.groupby('station', as_index=False)['indicator'].mean()
station_indicator_avg = station_indicator_avg.sort_values(by='indicator', ascending=False)
# 막대그래프 시각화
fig = px.bar(
    station_indicator_avg,
    x='station',
    y='indicator',
    title='지하철역별 놀거리 평가지수 평균',
    labels={'station': '지하철역', 'indicator': '평균 지표값'},
    color_discrete_sequence=['green']
)

fig.show()

In [31]:
# 상위 10개만 추출
indicator_top10 = station_indicator_avg.head(10)

# 그래프 그리기
fig = px.bar(
    indicator_top10,
    x='station',
    y='indicator',
    title='평균 지표값 기준 상위 10개 지하철역',
    color_discrete_sequence=['red']

)
fig.update_layout(
    width=800,
    height=500,
    bargap=0.3,  # 바 간격 (막대 폭 줄이기 효과)
    title='놀거리 평균 지표값 기준 상위 10개 지하철역',
    xaxis_title='지하철역',
    yaxis_title='평균 지표값'
)
fig.show()

In [33]:
# value 오름차순 정렬 (작은 값이 먼저 오게)
station_avg_indicator_sorted = station_indicator_avg.sort_values(by='indicator', ascending=True)

# 하위 10개만 추출
indicator_bottom10 = station_avg_indicator_sorted.head(10)

# 그래프 그리기
fig = px.bar(
    indicator_bottom10,
    x='station',
    y='indicator',
    title='놀거리 평균 지표값 기준 하위 10개 지하철역',
    color_discrete_sequence=['blue']
)

fig.update_layout(
    width=800,
    height=500,
    bargap=0.3,
    xaxis_title='지하철역',
    yaxis_title='평균 지표값'
)

fig.show()